## 1. Setup and Load Data

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sentence_transformers import SentenceTransformer
import torch
from sklearn.metrics.pairwise import cosine_similarity

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

DATA_DIR = Path('data_new')
MODELS_DIR = Path('models')

# Load data
df = pd.read_csv(DATA_DIR / 'SNow_incident_ticket_data.csv')
print(f"Loaded {len(df)} incidents")

In [ ]:
# Recreate combined text (same logic as evaluate_model.ipynb)
def create_combined_text(row):
    text_parts = []
    for col in ['Number', 'Description', 'User input', 'Resolution notes']:
        if col in row.index:
            value = str(row.get(col, '')).strip() if pd.notna(row.get(col)) else ''
            if value and value.lower() != 'nan':
                text_parts.append(value)
    return ' '.join(text_parts) if text_parts else ''

df['combined_text'] = df.apply(create_combined_text, axis=1)
df['combined_text'] = df['combined_text'].astype(str)
df = df[df['combined_text'].str.len() > 10].reset_index(drop=True)

print(f"After filtering: {len(df)} valid incidents")
print(f"\nData columns: {df.columns.tolist()}")
print(f"Has 'Category' column: {'Category' in df.columns}")

## 2. Generate Test Pairs (Same as evaluate_model.ipynb)

In [ ]:
from typing import List, Tuple
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity as sklearn_cosine_similarity

def generate_test_pairs(df: pd.DataFrame, 
                       num_positives: int = 500,
                       num_easy_negatives: int = 500,
                       num_hard_negatives: int = 500,
                       random_state: int = 42) -> Tuple[List[str], List[str], List[int]]:
    
    np.random.seed(random_state)
    texts1, texts2, labels = [], [], []
    has_categories = 'Category' in df.columns
    
    if has_categories:
        print("Using category-based pair generation")
        categories = df['Category'].dropna().unique()
        print(f"Found {len(categories)} categories")
        
        # Positive pairs - same category
        for _ in range(num_positives):
            cat = np.random.choice(categories)
            cat_incidents = df[df['Category'] == cat]
            if len(cat_incidents) >= 2:
                idx1, idx2 = np.random.choice(cat_incidents.index, size=2, replace=False)
                texts1.append(df.loc[idx1, 'combined_text'])
                texts2.append(df.loc[idx2, 'combined_text'])
                labels.append(1)
        
        # Easy negatives - different categories
        for _ in range(num_easy_negatives):
            cat1, cat2 = np.random.choice(categories, size=2, replace=False)
            incidents1 = df[df['Category'] == cat1]
            incidents2 = df[df['Category'] == cat2]
            if len(incidents1) > 0 and len(incidents2) > 0:
                idx1 = np.random.choice(incidents1.index)
                idx2 = np.random.choice(incidents2.index)
                texts1.append(df.loc[idx1, 'combined_text'])
                texts2.append(df.loc[idx2, 'combined_text'])
                labels.append(0)
    else:
        print("No category column found - using random pair generation")
        for _ in range(num_positives):
            idx1, idx2 = np.random.choice(len(df), size=2, replace=False)
            texts1.append(df.loc[idx1, 'combined_text'])
            texts2.append(df.loc[idx2, 'combined_text'])
            labels.append(1)
        for _ in range(num_easy_negatives):
            idx1, idx2 = np.random.choice(len(df), size=2, replace=False)
            texts1.append(df.loc[idx1, 'combined_text'])
            texts2.append(df.loc[idx2, 'combined_text'])
            labels.append(0)
    
    # Hard negatives
    print("\nGenerating hard negatives with TF-IDF...")
    sample_size = min(2000, len(df))
    sample_df = df.sample(n=sample_size, random_state=random_state)
    
    vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(sample_df['combined_text'])
    
    hard_neg_count = 0
    attempts = 0
    max_attempts = num_hard_negatives * 10
    
    while hard_neg_count < num_hard_negatives and attempts < max_attempts:
        idx1 = np.random.randint(0, len(sample_df))
        similarities = sklearn_cosine_similarity(tfidf_matrix[idx1:idx1+1], tfidf_matrix).flatten()
        hard_neg_candidates = np.where((similarities > 0.3) & (similarities < 0.6))[0]
        hard_neg_candidates = hard_neg_candidates[hard_neg_candidates != idx1]
        
        if len(hard_neg_candidates) > 0:
            idx2 = np.random.choice(hard_neg_candidates)
            if has_categories:
                cat1 = sample_df.iloc[idx1].get('Category')
                cat2 = sample_df.iloc[idx2].get('Category')
                if cat1 == cat2:
                    attempts += 1
                    continue
            
            texts1.append(sample_df.iloc[idx1]['combined_text'])
            texts2.append(sample_df.iloc[idx2]['combined_text'])
            labels.append(0)
            hard_neg_count += 1
        
        attempts += 1
    
    print(f"Generated {hard_neg_count} hard negatives ({attempts} attempts)")
    print(f"\nTotal test pairs: {len(labels)}")
    print(f"  Positive pairs: {sum(labels)}")
    print(f"  Negative pairs: {len(labels) - sum(labels)}")
    
    return texts1, texts2, labels

# Generate pairs
test_texts1, test_texts2, test_labels = generate_test_pairs(df, random_state=42)
test_labels = np.array(test_labels)

## 3. Inspect Test Pair Quality

In [ ]:
# Show examples of positive pairs
print("="*80)
print("POSITIVE PAIR EXAMPLES (Label=1 - Should be similar)")
print("="*80)

pos_indices = np.where(test_labels == 1)[0]
for i in range(min(3, len(pos_indices))):
    idx = pos_indices[i]
    print(f"\n--- Positive Pair {i+1} ---")
    print(f"Text 1 ({len(test_texts1[idx])} chars):")
    print(test_texts1[idx][:200] + "..." if len(test_texts1[idx]) > 200 else test_texts1[idx])
    print(f"\nText 2 ({len(test_texts2[idx])} chars):")
    print(test_texts2[idx][:200] + "..." if len(test_texts2[idx]) > 200 else test_texts2[idx])

In [ ]:
# Show examples of negative pairs
print("\n" + "="*80)
print("NEGATIVE PAIR EXAMPLES (Label=0 - Should be different)")
print("="*80)

neg_indices = np.where(test_labels == 0)[0]
for i in range(min(3, len(neg_indices))):
    idx = neg_indices[i]
    print(f"\n--- Negative Pair {i+1} ---")
    print(f"Text 1 ({len(test_texts1[idx])} chars):")
    print(test_texts1[idx][:200] + "..." if len(test_texts1[idx]) > 200 else test_texts1[idx])
    print(f"\nText 2 ({len(test_texts2[idx])} chars):")
    print(test_texts2[idx][:200] + "..." if len(test_texts2[idx]) > 200 else test_texts2[idx])

## 4. Calculate Baseline Model Embeddings

In [ ]:
# Load baseline model
print("Loading baseline model...")
baseline_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device=device)

# Encode test pairs
print("Encoding text pairs...")
embeddings1_baseline = baseline_model.encode(test_texts1, batch_size=64, show_progress_bar=True, convert_to_numpy=True)
embeddings2_baseline = baseline_model.encode(test_texts2, batch_size=64, show_progress_bar=True, convert_to_numpy=True)

# Compute similarity scores
baseline_scores = np.sum(embeddings1_baseline * embeddings2_baseline, axis=1) / (
    np.linalg.norm(embeddings1_baseline, axis=1) * np.linalg.norm(embeddings2_baseline, axis=1)
)

print(f"\nBaseline scores: min={baseline_scores.min():.4f}, max={baseline_scores.max():.4f}, mean={baseline_scores.mean():.4f}")

## 5. Analyze Score Distributions

In [ ]:
# Separate by label
pos_scores_baseline = baseline_scores[test_labels == 1]
neg_scores_baseline = baseline_scores[test_labels == 0]

print("="*80)
print("BASELINE MODEL SCORE DISTRIBUTION")
print("="*80)
print(f"\nPositive pairs (label=1):")
print(f"  Mean: {pos_scores_baseline.mean():.4f}")
print(f"  Median: {np.median(pos_scores_baseline):.4f}")
print(f"  Std: {pos_scores_baseline.std():.4f}")
print(f"  Min-Max: [{pos_scores_baseline.min():.4f}, {pos_scores_baseline.max():.4f}]")

print(f"\nNegative pairs (label=0):")
print(f"  Mean: {neg_scores_baseline.mean():.4f}")
print(f"  Median: {np.median(neg_scores_baseline):.4f}")
print(f"  Std: {neg_scores_baseline.std():.4f}")
print(f"  Min-Max: [{neg_scores_baseline.min():.4f}, {neg_scores_baseline.max():.4f}]")

# Overlap analysis
overlap = np.sum((pos_scores_baseline < neg_scores_baseline.max()) & (pos_scores_baseline > neg_scores_baseline.min())) / len(pos_scores_baseline)
print(f"\nScore overlap (% of positives within negative score range): {overlap*100:.1f}%")

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histograms
ax = axes[0]
ax.hist(neg_scores_baseline, bins=50, alpha=0.6, label='Negative (label=0)', color='red', edgecolor='black')
ax.hist(pos_scores_baseline, bins=50, alpha=0.6, label='Positive (label=1)', color='green', edgecolor='black')
ax.set_xlabel('Cosine Similarity Score')
ax.set_ylabel('Frequency')
ax.set_title('Baseline Model: Score Distributions')
ax.legend()
ax.grid(True, alpha=0.3)

# Box plots
ax = axes[1]
data_to_plot = [neg_scores_baseline, pos_scores_baseline]
bp = ax.boxplot(data_to_plot, labels=['Negative (0)', 'Positive (1)'], patch_artist=True)
bp['boxes'][0].set_facecolor('red')
bp['boxes'][0].set_alpha(0.6)
bp['boxes'][1].set_facecolor('green')
bp['boxes'][1].set_alpha(0.6)
ax.set_ylabel('Cosine Similarity Score')
ax.set_title('Baseline Model: Score Distributions (Box Plot)')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\nSeparability: Mean positive ({pos_scores_baseline.mean():.4f}) vs Mean negative ({neg_scores_baseline.mean():.4f})")
print(f"Difference: {pos_scores_baseline.mean() - neg_scores_baseline.mean():.4f}")

## 6. Analyze Hard Negatives - Are They Actually Hard?

In [ ]:
# Identify which negatives are "hard" (high TF-IDF similarity)
# We'll use a separate baseline encoding to compute similarity

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_matrix = vectorizer.fit_transform([t for t in test_texts1 + test_texts2])

# For simplicity, estimate which negatives are hard by checking text similarity
print("Analyzing negative pairs for semantic overlap...\n")

neg_indices = np.where(test_labels == 0)[0]
neg_example_similarities = []

for idx in neg_indices[:100]:  # Sample first 100 negatives
    # Simple word overlap ratio
    words1 = set(test_texts1[idx].lower().split())
    words2 = set(test_texts2[idx].lower().split())
    overlap = len(words1 & words2) / (len(words1 | words2) + 1e-10)
    neg_example_similarities.append(overlap)

print(f"Negative pair word overlap (sample of 100):")
print(f"  Mean: {np.mean(neg_example_similarities):.4f}")
print(f"  Median: {np.median(neg_example_similarities):.4f}")
print(f"  Max: {np.max(neg_example_similarities):.4f}")
print(f"\nThis tells us if 'different category' pairs actually share significant vocabulary...")

## 7. Check for Label Noise in Positive Pairs

In [ ]:
# For positive pairs labeled as "same category", check if they're actually similar
print("Analyzing positive pairs for semantic similarity...\n")

pos_indices = np.where(test_labels == 1)[0]
pos_example_similarities = []

for idx in pos_indices[:100]:  # Sample first 100 positives
    words1 = set(test_texts1[idx].lower().split())
    words2 = set(test_texts2[idx].lower().split())
    overlap = len(words1 & words2) / (len(words1 | words2) + 1e-10)
    pos_example_similarities.append(overlap)

print(f"Positive pair word overlap (sample of 100):")
print(f"  Mean: {np.mean(pos_example_similarities):.4f}")
print(f"  Median: {np.median(pos_example_similarities):.4f}")
print(f"  Min: {np.min(pos_example_similarities):.4f}")
print(f"\nProblem detected if mean overlap is similar to negative pairs!")
print(f"\nComparison:")
print(f"  Positive pairs mean overlap: {np.mean(pos_example_similarities):.4f}")
print(f"  Negative pairs mean overlap: {np.mean(neg_example_similarities):.4f}")
print(f"  Difference: {np.mean(pos_example_similarities) - np.mean(neg_example_similarities):.4f}")

## 8. Visualize Pair Quality Issues

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Positive pairs
ax = axes[0]
ax.hist(pos_example_similarities, bins=30, color='green', alpha=0.7, edgecolor='black')
ax.set_xlabel('Word Overlap Ratio')
ax.set_ylabel('Frequency')
ax.set_title('Positive Pairs: Word Overlap Distribution\n(Same Category - Should Be High)')
ax.axvline(np.mean(pos_example_similarities), color='darkgreen', linestyle='--', linewidth=2, label=f'Mean: {np.mean(pos_example_similarities):.3f}')
ax.legend()
ax.grid(True, alpha=0.3)

# Negative pairs
ax = axes[1]
ax.hist(neg_example_similarities, bins=30, color='red', alpha=0.7, edgecolor='black')
ax.set_xlabel('Word Overlap Ratio')
ax.set_ylabel('Frequency')
ax.set_title('Negative Pairs: Word Overlap Distribution\n(Different Category - Should Be Low)')
ax.axvline(np.mean(neg_example_similarities), color='darkred', linestyle='--', linewidth=2, label=f'Mean: {np.mean(neg_example_similarities):.3f}')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Diagnose: Key Metrics

In [ ]:
print("\n" + "="*80)
print("DIAGNOSIS: Why Fine-Tuned Models Underperform")
print("="*80)

# Check 1: Score separability
separability = pos_scores_baseline.mean() - neg_scores_baseline.mean()
print(f"\n1. BASELINE MODEL SEPARABILITY")
print(f"   Positive mean: {pos_scores_baseline.mean():.4f}")
print(f"   Negative mean: {neg_scores_baseline.mean():.4f}")
print(f"   Difference: {separability:.4f}")
if separability < 0.1:
    print(f"   ⚠️  WARNING: Poor separability! Scores barely different.")
else:
    print(f"   ✓ Good separability")

# Check 2: Overlap
print(f"\n2. SCORE DISTRIBUTION OVERLAP")
print(f"   % of positive scores < mean negative score: {(pos_scores_baseline < neg_scores_baseline.mean()).sum()/len(pos_scores_baseline)*100:.1f}%")
print(f"   % of negative scores > mean positive score: {(neg_scores_baseline > pos_scores_baseline.mean()).sum()/len(neg_scores_baseline)*100:.1f}%")
if overlap > 0.5:
    print(f"   ⚠️  WARNING: High overlap suggests noisy labels!")

# Check 3: Word overlap in pairs
print(f"\n3. SEMANTIC OVERLAP IN PAIRS")
print(f"   Positive pairs word overlap: {np.mean(pos_example_similarities):.4f}")
print(f"   Negative pairs word overlap: {np.mean(neg_example_similarities):.4f}")
print(f"   Difference: {np.mean(pos_example_similarities) - np.mean(neg_example_similarities):.4f}")
if abs(np.mean(pos_example_similarities) - np.mean(neg_example_similarities)) < 0.05:
    print(f"   ⚠️  WARNING: Positive and negative pairs are too similar!")
else:
    print(f"   ✓ Good semantic separation")

# Check 4: Category-based label validation
print(f"\n4. TEST SET COMPOSITION")
print(f"   Total pairs: {len(test_labels)}")
print(f"   Positive pairs: {sum(test_labels)} ({sum(test_labels)/len(test_labels)*100:.1f}%)")
print(f"   Negative pairs: {len(test_labels)-sum(test_labels)} ({(len(test_labels)-sum(test_labels))/len(test_labels)*100:.1f}%)")
print(f"   Expected ratio: 50-50 for balanced test set")

print(f"\n" + "="*80)
print("RECOMMENDATIONS:")
print("="*80)
if separability < 0.1:
    print("✗ Low baseline separability → Test set may have noisy/weak labels")
    print("  ACTION: Review 'positive' pair labels - same category ≠ semantically similar")
if overlap > 0.5:
    print("✗ High score overlap → Hard to distinguish positives from negatives")
    print("  ACTION: Use stricter pair generation (require min semantic similarity for positives)")
print("\nAll models (baseline + fine-tuned) struggle with this test set.")
print("The problem is likely in test pair generation, not model quality.")